In [3]:
pip install tensorflow

     |████████████████████████████████| 458.3 MB 18 kB/s s eta 0:00:01     |████████████▊                   | 182.6 MB 75.2 MB/s eta 0:00:04��█████████████▎              | 247.1 MB 79.1 MB/s eta 0:00:03��█████████████▌              | 251.0 MB 79.1 MB/s eta 0:00:03     |██████████████████████████████▎ | 434.0 MB 66.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 61.8 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 61.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 61.4 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 81.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 274 kB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 64.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.8 MB/s  eta 0:00:01
     |████████████████████████████████| 97 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 152 kB 82.7 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 

In [6]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import tensorflow as tf

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [38]:
## Defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [41]:
## Transforming the input variables
scaler_train = MinMaxScaler(feature_range = (0, 1)).fit(X_train)
X_train = scaler_train.transform(X_train)

scaler_test = MinMaxScaler(feature_range = (0, 1)).fit(X_test)
X_test = scaler_train.transform(X_test)

In [51]:
mlp = tf.keras.models.Sequential([
      tf.keras.layers.Dense(4, input_dim = 5, activation = 'tanh'),
      tf.keras.layers.Dense(2, activation = 'softmax')
])

mlp.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Building the model
md_mlp = mlp.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 2), 
                 epochs = 100,
                 batch_size = 32, 
                 validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)))

Epoch 1/100
92/92 [==============================] - 0s 3ms/step - loss: 0.5117 - accuracy: 0.8376 - val_loss: 0.4695 - val_accuracy: 0.8374
Epoch 2/100
92/92 [==============================] - 0s 1ms/step - loss: 0.4379 - accuracy: 0.8502 - val_loss: 0.4483 - val_accuracy: 0.8374
Epoch 3/100
92/92 [==============================] - 0s 1ms/step - loss: 0.4250 - accuracy: 0.8502 - val_loss: 0.4446 - val_accuracy: 0.8374
Epoch 4/100
92/92 [==============================] - 0s 1ms/step - loss: 0.4219 - accuracy: 0.8502 - val_loss: 0.4438 - val_accuracy: 0.8374
Epoch 5/100
92/92 [==============================] - 0s 1ms/step - loss: 0.4208 - accuracy: 0.8502 - val_loss: 0.4435 - val_accuracy: 0.8374
Epoch 6/100
92/92 [==============================] - 0s 1ms/step - loss: 0.4200 - accuracy: 0.8502 - val_loss: 0.4432 - val_accuracy: 0.8374
Epoch 7/100
92/92 [==============================] - 0s 1ms/step - loss: 0.4194 - accuracy: 0.8502 - val_loss: 0.4428 - val_accuracy: 0.8374
Epoch 8/100
9

In [53]:
## Extracting predictions
mlp_pred = mlp.predict(X_test)[:, 1]
mlp_pred

array([0.15323302, 0.06984502, 0.13597395, 0.2146586 , 0.1560349 ,
       0.07139845, 0.07139841, 0.31990385, 0.05781847, 0.30887046,
       0.21517548, 0.10266523, 0.1052977 , 0.17609064, 0.10253438,
       0.0804897 , 0.1426054 , 0.1391601 , 0.05057511, 0.08691587,
       0.06458239, 0.10722248, 0.0787853 , 0.30917954, 0.07386761,
       0.1006656 , 0.10923968, 0.21782225, 0.07541741, 0.0901816 ,
       0.08316138, 0.11798892, 0.14070423, 0.1586984 , 0.21846287,
       0.15870616, 0.17795467, 0.08347883, 0.25167057, 0.16057757,
       0.26159593, 0.11303103, 0.0910034 , 0.08102007, 0.06173062,
       0.30791318, 0.14463773, 0.07282171, 0.21666066, 0.20883271,
       0.08741899, 0.18231405, 0.1779998 , 0.1743781 , 0.17675932,
       0.2107315 , 0.10983691, 0.18581906, 0.3011414 , 0.10917295,
       0.13679817, 0.10683965, 0.24742593, 0.0987094 , 0.32997146,
       0.0880011 , 0.17003399, 0.11081491, 0.06492715, 0.13743582,
       0.26303503, 0.16964886, 0.06806266, 0.04839081, 0.06057